In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import random; random.seed(10)
from scipy import stats
from scipy.stats import norm, chi2, f
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
from statsmodels.graphics import regressionplots
from statsmodels.stats.multicomp import MultiComparison
from statsmodels.robust.robust_linear_model import RLM
from statsmodels.iolib.summary2 import summary_col
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import linear_reset
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy
from statsmodels.stats.diagnostic import het_breuschpagan, het_white, het_goldfeldquandt
from sklearn.impute import KNNImputer
import morethemes as mt
mt.set_theme('economist')

In [27]:
ROP = pd.read_csv('data/ROP1.csv')
ROP = ROP.rename(columns={'observation_date': 'Date'})
ROP['Date'] = pd.to_datetime(ROP['Date'])
ROP['Date'] = ROP['Date'].dt.year.astype(str) + 'Q' + ROP['Date'].dt.quarter.astype(str)
ROP = ROP.set_index('Date')
ROP = ROP.rename(columns={'B471RC1Q027SBEA': 'ROP'})
ROP

,ROP
Date,
1947Q1,0.767000
1947Q2,0.767000
1947Q3,0.768000
1947Q4,0.768000
1948Q1,0.803000
...,...
2024Q2,199.358000
2024Q3,199.407000
2024Q4,199.348000


In [28]:
df = pd.read_csv('data/master_quarterly.csv')
df = df.rename(columns={'Unnamed: 0': 'Date'})
df = df.set_index('Date')
df['ROP'] = ROP['ROP']
df['CGDP_change'] = df['CGDP'].diff()
df[['CGDP', 'CGDP_change']].head(10)
df['HP_change'] = df['HP'].diff()
df['IR_change'] = df['IR'].diff()
df['LSOI_change'] = df['LSOI'].diff()
df['INVR'] = df['INVR']/100
df['IROR'] = df['IROR']*1000000
df = df.dropna()
df

,R,CGDP,CPIR,INVR,HP,IR,ROP,IROR,LSOI,RGDP,RGDP.1,MA,TOBQ,CGDP_change,HP_change,IR_change,LSOI_change
Date,,,,,,,,,,,,,,,,,
1970Q3,1.000000,90.600000,1.035375,0.007124,1.561012,5.294625,18.358000,-16.431211,113.180000,5357.077000,0.921467,44.100000,0.629600,0.400000,0.828463,0.014450,-0.520000
1970Q4,0.000000,91.400000,1.451751,-0.025575,1.263789,3.898249,18.701000,-64.103580,113.699000,5299.672000,-1.071573,45.400000,0.684200,0.800000,-0.297223,-1.396375,0.519000
1971Q1,0.000000,90.900000,0.841751,0.016999,2.694829,2.998249,18.799000,126.052571,111.541000,5443.619000,2.716149,51.200000,0.734200,-0.500000,1.431040,-0.900000,-2.158000
1971Q2,0.000000,91.800000,0.918197,0.022017,2.675118,3.331803,18.849000,0.568634,111.502000,5473.059000,0.540817,53.800000,0.720300,0.900000,-0.019711,0.333554,-0.039000
1971Q3,0.000000,92.100000,0.992556,0.012404,1.080599,4.017444,18.937000,7.802701,110.939000,5518.072000,0.822447,55.100000,0.691500,0.300000,-1.594519,0.685641,-0.563000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017Q2,0.000000,153.400000,0.115377,0.020029,1.413681,0.774623,351.937000,5.970152,100.101000,19506.949000,0.559873,57.800000,1.264000,0.700000,0.075611,0.888364,0.380000
2017Q3,0.000000,153.400000,0.478044,0.010460,1.516397,0.561956,351.798000,-21.769587,100.062000,19660.766000,0.788524,60.800000,1.338000,0.000000,0.102717,-0.212667,-0.039000
2017Q4,0.000000,154.100000,0.795531,0.026467,1.587257,0.414469,351.852000,22.488287,100.127000,19882.352000,1.127047,59.300000,1.364000,0.700000,0.070859,-0.147487,0.065000


In [29]:
model1 = (smf.probit('R ~ CGDP_change + CPIR + MA + INVR '
                     '+ HP_change + IR_change', df)
          .fit(data=df))
print(model1.summary())

Optimization terminated successfully.
         Current function value: 0.202308
         Iterations 8
                          Probit Regression Results                           
Dep. Variable:                      R   No. Observations:                  192
Model:                         Probit   Df Residuals:                      185
Method:                           MLE   Df Model:                            6
Date:                Tue, 02 Dec 2025   Pseudo R-squ.:                  0.4630
Time:                        19:58:31   Log-Likelihood:                -38.843
converged:                       True   LL-Null:                       -72.340
Covariance Type:            nonrobust   LLR p-value:                 1.688e-12
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       4.0362      1.395      2.894      0.004       1.303       6.770
CGDP_change     0.0424    

/opt/anaconda3/lib/python3.13/site-packages/statsmodels/base/optimizer.py:21: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol, ridge_factor. The list of unsupported keyword arguments passed include: data. After release 0.14, this will raise.
  warnings.warn(


In [30]:
model2 = (smf.probit('R ~ CGDP_change + CPIR + MA + INVR '
                     '+ HP_change + ROP', df)
          .fit(data=df))
print(model2.summary())

Optimization terminated successfully.
         Current function value: 0.201925
         Iterations 8
                          Probit Regression Results                           
Dep. Variable:                      R   No. Observations:                  192
Model:                         Probit   Df Residuals:                      185
Method:                           MLE   Df Model:                            6
Date:                Tue, 02 Dec 2025   Pseudo R-squ.:                  0.4641
Time:                        19:58:31   Log-Likelihood:                -38.770
converged:                       True   LL-Null:                       -72.340
Covariance Type:            nonrobust   LLR p-value:                 1.575e-12
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       3.1864      1.362      2.340      0.019       0.517       5.856
CGDP_change     0.0869    

/opt/anaconda3/lib/python3.13/site-packages/statsmodels/base/optimizer.py:21: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol, ridge_factor. The list of unsupported keyword arguments passed include: data. After release 0.14, this will raise.
  warnings.warn(


In [31]:
model3 = (smf.probit('R ~ CGDP_change + CPIR + MA + INVR '
                     '+ HP_change + IROR', df)
          .fit(data=df))
print(model3.summary())

Optimization terminated successfully.
         Current function value: 0.198741
         Iterations 8
                          Probit Regression Results                           
Dep. Variable:                      R   No. Observations:                  192
Model:                         Probit   Df Residuals:                      185
Method:                           MLE   Df Model:                            6
Date:                Tue, 02 Dec 2025   Pseudo R-squ.:                  0.4725
Time:                        19:58:32   Log-Likelihood:                -38.158
converged:                       True   LL-Null:                       -72.340
Covariance Type:            nonrobust   LLR p-value:                 8.853e-13
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       3.1586      1.402      2.254      0.024       0.412       5.906
CGDP_change     0.0800    

/opt/anaconda3/lib/python3.13/site-packages/statsmodels/base/optimizer.py:21: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol, ridge_factor. The list of unsupported keyword arguments passed include: data. After release 0.14, this will raise.
  warnings.warn(


In [32]:
model4 = (smf.probit('R ~ CGDP_change + CPIR + MA + INVR '
                     '+ HP_change + LSOI_change', df)
          .fit(data=df))
print(model4.summary())

Optimization terminated successfully.
         Current function value: 0.208674
         Iterations 8
                          Probit Regression Results                           
Dep. Variable:                      R   No. Observations:                  192
Model:                         Probit   Df Residuals:                      185
Method:                           MLE   Df Model:                            6
Date:                Tue, 02 Dec 2025   Pseudo R-squ.:                  0.4461
Time:                        19:58:32   Log-Likelihood:                -40.065
converged:                       True   LL-Null:                       -72.340
Covariance Type:            nonrobust   LLR p-value:                 5.333e-12
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       3.6446      1.346      2.708      0.007       1.006       6.283
CGDP_change     0.0855    

/opt/anaconda3/lib/python3.13/site-packages/statsmodels/base/optimizer.py:21: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol, ridge_factor. The list of unsupported keyword arguments passed include: data. After release 0.14, this will raise.
  warnings.warn(


In [33]:
model5 = (smf.probit('R ~ CGDP_change + CPIR + MA + INVR '
                     '+ HP_change + TOBQ', df)
          .fit(data=df))
print(model5.summary())

Optimization terminated successfully.
         Current function value: 0.210871
         Iterations 8
                          Probit Regression Results                           
Dep. Variable:                      R   No. Observations:                  192
Model:                         Probit   Df Residuals:                      185
Method:                           MLE   Df Model:                            6
Date:                Tue, 02 Dec 2025   Pseudo R-squ.:                  0.4403
Time:                        19:58:33   Log-Likelihood:                -40.487
converged:                       True   LL-Null:                       -72.340
Covariance Type:            nonrobust   LLR p-value:                 7.927e-12
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       3.7299      1.347      2.768      0.006       1.089       6.370
CGDP_change     0.0816    

/opt/anaconda3/lib/python3.13/site-packages/statsmodels/base/optimizer.py:21: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol, ridge_factor. The list of unsupported keyword arguments passed include: data. After release 0.14, this will raise.
  warnings.warn(


In [34]:
coef_dict = {}

for i, model in enumerate([model1, model2, model3, model4, model5], 
                          start=1):
    coef_dict[f'Model {i}'] = model.params

coef_table = pd.DataFrame(coef_dict)

if 'Intercept' in coef_table.index:
    coef_table = coef_table.rename(index={'Intercept': 'Constant'})

order = ['CGDP_change', 'CPIR', 'MA', 'INVR', 'HP_change', 
         'IR_change', 'ROP', 'IROR', 'LSOI_change', 'TOBQ', 
         'Constant']
ordered_index = [var for var in order if var in coef_table.index]
coef_table = coef_table.reindex(ordered_index)

coef_table

,Model 1,Model 2,Model 3,Model 4,Model 5
CGDP_change,0.042399,0.086888,0.079989,0.085499,0.081628
CPIR,0.707364,0.983832,0.808460,0.791329,0.777475
MA,-0.119652,-0.119307,-0.104121,-0.111480,-0.114589
INVR,-16.089632,-18.274907,-20.033631,-22.071476,-19.815263
HP_change,-0.418165,-0.346102,-0.427768,-0.364731,-0.363364
IR_change,-0.311307,NaN,NaN,NaN,NaN
ROP,NaN,0.003692,NaN,NaN,NaN
IROR,NaN,NaN,-0.009269,NaN,NaN
LSOI_change,NaN,NaN,NaN,0.136152,NaN
TOBQ,NaN,NaN,NaN,NaN,0.079679


In [35]:
me_dict = {}

for i, model in enumerate([model1, model2, model3, model4, model5], 
                          start=1):
    me = model.get_margeff()
    me_summary = me.summary_frame()
    me_dict[f'Model {i}'] = me_summary['dy/dx']

me_table = pd.DataFrame(me_dict)

order = ['CGDP_change', 'CPIR', 'MA', 'INVR', 'HP_change', 
         'IR_change', 'ROP', 'IROR', 'LSOI_change', 'TOBQ']
ordered_index = [var for var in order if var in me_table.index]
me_table = me_table.reindex(ordered_index)

coef_separator = pd.DataFrame([['Coefficients'] * 5], 
                              columns=coef_table.columns, 
                              index=[''])
me_separator = pd.DataFrame([['Marginal Effects'] * 5], 
                            columns=me_table.columns, 
                            index=[''])

combined_table = pd.concat([coef_separator, coef_table, 
                           me_separator, me_table])

combined_table

,Model 1,Model 2,Model 3,Model 4,Model 5
,Coefficients,Coefficients,Coefficients,Coefficients,Coefficients
CGDP_change,0.042399,0.086888,0.079989,0.085499,0.081628
CPIR,0.707364,0.983832,0.808460,0.791329,0.777475
MA,-0.119652,-0.119307,-0.104121,-0.111480,-0.114589
INVR,-16.089632,-18.274907,-20.033631,-22.071476,-19.815263
HP_change,-0.418165,-0.346102,-0.427768,-0.364731,-0.363364
IR_change,-0.311307,NaN,NaN,NaN,NaN
ROP,NaN,0.003692,NaN,NaN,NaN
IROR,NaN,NaN,-0.009269,NaN,NaN
LSOI_change,NaN,NaN,NaN,0.136152,NaN


In [ ]:
# Create a table of model statistics
stats_dict = {}

for i, model in enumerate([model1, model2, model3, model4, model5], 
                          start=1):
    # Get predictions
    predictions = model.predict()
    actual = model.model.endog
    
    # Count actual recessions and non-recessions
    n_actual_recession = (actual == 1).sum()
    n_actual_non_recession = (actual == 0).sum()
    n_total = len(actual)
    
    # 50% threshold
    pred_50 = (predictions >= 0.5).astype(int)
    correct_recession_50 = ((pred_50 == 1) & (actual == 1)).sum() / n_actual_recession
    correct_non_recession_50 = ((pred_50 == 0) & (actual == 0)).sum() / n_actual_non_recession
    correct_overall_50 = (pred_50 == actual).sum() / n_total
    
    # 12.16% threshold
    pred_1216 = (predictions >= 0.1216).astype(int)
    correct_recession_1216 = ((pred_1216 == 1) & (actual == 1)).sum() / n_actual_recession
    correct_non_recession_1216 = ((pred_1216 == 0) & (actual == 0)).sum() / n_actual_non_recession
    correct_overall_1216 = (pred_1216 == actual).sum() / n_total
    
    stats_dict[f'Model {i}'] = {
        'Observations': int(model.nobs),
        'Pseudo R²': model.prsquared,
        'AIC': model.aic,
        'BIC': model.bic,
        'Log-Likelihood': model.llf,
        'LLR p-value': model.llr_pvalue,
        'Correct Recession (50%)': correct_recession_50,
        'Correct Non-Recession (50%)': correct_non_recession_50,
        'Correct Overall (50%)': correct_overall_50,
        'Correct Recession (12.16%)': correct_recession_1216,
        'Correct Non-Recession (12.16%)': correct_non_recession_1216,
        'Correct Overall (12.16%)': correct_overall_1216
    }

stats_table = pd.DataFrame(stats_dict).T
stats_table = stats_table.T

# Format the table to avoid scientific notation
pd.options.display.float_format = '{:.6f}'.format

stats_table

,Model 1,Model 2,Model 3,Model 4,Model 5
Observations,192.000000,192.000000,192.000000,192.000000,192.000000
Pseudo R²,0.463046,0.464064,0.472514,0.446150,0.440319
AIC,91.686322,91.539050,90.316593,94.130885,94.974496
BIC,114.488789,114.341517,113.119061,116.933352,117.776964
Log-Likelihood,-38.843161,-38.769525,-38.158297,-40.065442,-40.487248
LLR p-value,0.000000,0.000000,0.000000,0.000000,0.000000
Correct Recession (50%),14.000000,12.000000,12.000000,11.000000,10.000000
Correct Non-Recession (50%),166.000000,165.000000,163.000000,165.000000,165.000000
Correct Overall (50%),180.000000,177.000000,175.000000,176.000000,175.000000
Correct Recession (12.16%),20.000000,21.000000,23.000000,22.000000,22.000000
